In [3]:
!pip install pandas
!pip install ortools
!pip install openpyxl
import pandas as pd
from ortools.linear_solver import pywraplp
import numpy as np

df = pd.read_excel("solved_data.xlsx")


#produccion
cup = pd.DataFrame(df.iloc[22-2:34-2, 2:7])
cup = cup.values
cpr = 1000


#horas extra
cuhe = 250
che = 0.335


#subcontratadas
csub = pd.DataFrame(df.iloc[22-2:34-2, 16:21])
csub = csub.values


#stock de seguridad
stock_seguridad = pd.DataFrame(df.iloc[6-2:18-2, 23:28])
stock_seguridad = stock_seguridad.values
css = pd.DataFrame(df.iloc[22-2:34-2, 23:28])
css = css.values


#setup
cs = pd.DataFrame(df.iloc[22-2:34-2, 30:35])
cs = cs.values
m=50000


demanda = pd.DataFrame(df.iloc[6-2:18-2, 44:49])
demanda = demanda.values


#inventario
cin = 250
cui = pd.DataFrame(df.iloc[23-2:35-2, 37:42])
cui = cui.values



#recursos
recursos = pd.DataFrame(df.iloc[23-2:34-2, 44:49])
recursos = recursos.values


#disponibilidad
disponibilidad = pd.DataFrame(df.iloc[23-2:34-2, 49])
disponibilidad = disponibilidad.values


j=0
t=0

solver = pywraplp.Solver.CreateSolver('SCIP')

Xjt = []
UHEjt = []
USjt = []
SETjt = []
Ijt = []
for j in range(5):
    row0 = []
    row1 = []
    row2 = []
    row3 = []
    row4 = []
    for t in range(12):
        x = solver.NumVar(0.0, solver.infinity(), f'x_{j}_{t}')
        uhe = solver.NumVar(0.0, solver.infinity(), f'uhe_{j}_{t}')
        us = solver.NumVar(0.0, solver.infinity(), f'us_{j}_{t}')
        set = solver.BoolVar(f'x_{j}_{t}')
        inv = solver.NumVar(0.0, solver.infinity(), f'inv_{j}_{t}')
        row0.append(x)
        row1.append(uhe)
        row2.append(us)
        row3.append(set)
        row4.append(inv)
    Xjt.append(row0)
    UHEjt.append(row1)
    USjt.append(row2)
    SETjt.append(row3)
    Ijt.append(row4)




#Restriccion 1
i=0
j=0
t=0
for i in range(11):
    for j in range(5):
        for t in range(12):
            recurso = float(recursos[i][j])
            disp = float(disponibilidad[i])
            solver.Add(recurso * Xjt[j][t] + recurso * UHEjt[j][t] <= disp)




#Restriccion 2
j=0
t=0
for j in range(5):
    for t in range(12):
        if (t==0):
            solver.Add(Xjt[j][t] + USjt[j][t] + UHEjt[j][t] + stock_seguridad[t][j] - demanda[t][j] == Ijt[j][t])
        else:
            solver.Add(Ijt[j][t-1] + Xjt[j][t] + USjt[j][t] + UHEjt[j][t] + stock_seguridad[t][j] - demanda[t][j] == Ijt[j][t])




#Restriccion 3
j=0
t=0
for j in range(5):
    for t in range(12):
        solver.Add(Xjt[j][t] <= cpr)




#Restriccion 4
j=0
t=0
for j in range(5):
    for t in range(12):
        solver.Add(Ijt[j][t] <= cin)

#Restriccion 5
j=0
t=0
for j in range(5):
    for t in range(12):
        solver.Add(UHEjt[j][t] <= cuhe)





#Restriccion 7
j=0
t=0
for j in range(5):
    for t in range(12):
        solver.Add(Xjt[j][t] * recursos[0][j] <= (UHEjt[j][t] * recursos[0][j]) + 291)

#Restriccion 8
j=0
t=0
for j in range(5):
    for t in range(12):
        solver.Add(Xjt[j][t] + UHEjt[j][t] + USjt[j][t] <= m*SETjt[j][t])






#Restriccion 10
j=0
t=0
for j in range(5):
    for t in range(12):
        solver.Add(Xjt[j][t] >= UHEjt[j][t] + USjt[j][t])


#
j=0
t=0
for j in range(5):
    for t in range(12):
        solver.Add(Xjt[j][t] <= demanda[t][j] - stock_seguridad[t][j])


j=0
t=0
for j in range(5):
    for t in range(12):
        solver.Add(Ijt[j][t] <= stock_seguridad[t][j])


j=0
t=0
for j in range(5):
    for t in range(12):
        solver.Add(Xjt[j][t] >= 0)


j=0
t=0
for j in range(5):
    for t in range(12):
        solver.Add(UHEjt[j][t] >= 0)


j=0
t=0
for j in range(5):
    for t in range(12):
        solver.Add(0 <= USjt[j][t])


j=0
t=0
for j in range(5):
    for t in range(12):
        solver.Add(0 <= SETjt[j][t])


objective = solver.Objective()
j=0
t=0
for j in range(5):
    for t in range(12):
        objective.SetCoefficient(Xjt[j][t], cup[t][j])
        objective.SetCoefficient(Ijt[j][t], cui[t][j])
        objective.SetCoefficient(UHEjt[j][t], che)
        objective.SetCoefficient(USjt[j][t], csub[t][j])
        objective.SetCoefficient(SETjt[j][t], cs[t][j])



costoSeg=0
j=0
t=0
for j in range(5):
    for t in range(12):
        costoSeg += stock_seguridad[t][j] * css[t][j]
costito = costoSeg
objective.SetOffset(costito)

objective.SetMinimization()

resultado = solver.Solve()
print(solver.Objective().Value())





You should consider upgrading via the 'C:\Users\osmar\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\osmar\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


409407.755


You should consider upgrading via the 'C:\Users\osmar\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.
C:\Users\osmar\AppData\Local\Temp\ipykernel_51580\3211394744.py:105: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  disp = float(disponibilidad[i])
